In [100]:
# use Brown corpus movie reviews
from nltk.corpus import movie_reviews as mr
import string
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
ps = LancasterStemmer()
stop = stopwords.words('english')
# document [0] are the words list, [1] is 'pos' or 'neg' review
documents = [([ps.stem(w.lower()) for w in mr.words(i) if w.lower() not in stop and w.lower() not in string.punctuation], i.split('/')[0]) for i in mr.fileids()]
# unique words
print('documents:', len(documents))

documents: 2000


In [108]:
# organize unique words and their counts
from collections import Counter
words = []
for d in documents:
    for w in d[0]:
        words.append(w)
word_counts = Counter(words)
words = []
for w in word_counts:
    if 100 > word_counts[w] > 10:
        words.append(w)
print('# words:', len(words))

# words: 4673


In [112]:
import numpy as np
import random

def create_feature_set(document, words):
    features = np.zeros(len(words))
    for word in documents:
        if word in words:
            index_value = words.index(word)
            features[index_value] += 1
    return features
            
def create_feature_sets_and_labels(documents, test_size = 0.15):
    features = []
    for d in documents:
        if d[1] == 'pos':
            features.append([create_feature_set(d[0], words),[1,0]])
        elif d[1] == 'neg':
            features.append([create_feature_set(d[0], words),[0,1]])

    random.shuffle(features)
    features = np.array(features)

    testing_size = int(test_size*len(features))

    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])

    return train_x,train_y,test_x,test_y


if __name__ == '__main__':
    train_x,train_y,test_x,test_y = create_feature_sets_and_labels(documents)


KeyboardInterrupt: 

In [110]:
print(create_feature_set(documents[0], words))

[ 0.  0.  0. ...,  0.  0.  0.]


In [56]:
import tensorflow as tf
import pickle
import numpy as np

train_x,train_y,test_x,test_y = create_feature_sets_and_labels(documents)

n_nodes_hl1 = 250
n_nodes_hl2 = 250

n_classes = 2
hm_epochs = 10

x = tf.placeholder('float')
y = tf.placeholder('float')

hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


# Nothing changes
def neural_network_model(data):

    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.sigmoid(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.sigmoid(l2)

    output = tf.matmul(l2,output_layer['weight']) + output_layer['bias']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.0001, beta1=0.9, beta2=0.999).minimize(cost)
    optimizer = tf.train.GradientDescentOptimizer(1).minimize(cost)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
  
        for epoch in range(hm_epochs):
            epoch_loss = 0
            last_cost = 0

            batch_x = np.array(train_x[0:len(train_x)])
            batch_y = np.array(train_y[0:len(train_x)])

            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            epoch_loss += c
            last_cost = c

            if (epoch% 2) == 0 and epoch > 1:
                print('Epoch', epoch+1, 'completed out of',hm_epochs,'cost:', last_cost)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

print(x.__dict__)
train_neural_network(x)

{'_value_index': 0, '_shape': TensorShape(None), '_dtype': tf.float32, '_handle_dtype': 0, '_handle_shape': unknown_rank: true
, '_op': <tensorflow.python.framework.ops.Operation object at 0x7fdad77105c0>, '_consumers': []}


ValueError: setting an array element with a sequence.